## Ford SoH EDA 

Création de ce notebokk car une décroissance élevé des soh des ford mach-e à été constatée.

In [ ]:
from transform.raw_results.ford_results import *
import plotly.express as px
from core.sql_utils import *

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b
            on b.id = vm.battery_id
            WHERE vm.oem_id = '437e8d1a-0200-41da-a06b-4400c4fa5720';"""), con)

In [ ]:
px.scatter(dbeaver_df, x='odometer', y='soh', color='vin')

In [ ]:
raw_res = get_results()

In [ ]:
dbeaver_df.vin.unique()

In [ ]:
px.scatter(raw_res[raw_res['vin']=='WF0TK1EM5MMA31333'], x='odometer', y='soh', color='vin')

In [ ]:
px.scatter(raw_res[raw_res['vin']=='WF0TK1EM5MMA31333'], x='date', y='soh', color='vin')

In [ ]:
px.scatter(raw_res[raw_res['vin']=='WF0TK1E73MMA04075'], x='date', y='soh', color='vin')

In [ ]:
from transform.processed_results.main import *
from transform.processed_results.config import *
def get_processed_results(brand:str) -> DF:
    logger.info(f"{'Processing ' + brand + ' results.':=^{50}}")
    results =  (
        raw_res
        # Some raw estimations may have inf values, this will make mask_out_outliers_by_interquartile_range and force_monotonic_decrease fail
        # So we replace them by NaNs.
        .assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan))
        .sort_values(["vin", "date"])
        .pipe(make_charge_levels_presentable)
        .eval(SOH_FILTER_EVAL_STRINGS[brand])
        .groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle, include_groups=False)
        .pipe(agg_results_by_update_frequency)
        # .groupby('vin', observed=True)
        # .apply(make_soh_presentable_per_vehicle, include_groups=False)
        # .reset_index(level=0)
        # #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        # .sort_values(["vin", "date"])
    )
    results["soh"] = results.groupby("vin", observed=True)["soh"].ffill()
    results["soh"] = results.groupby("vin", observed=True)["soh"].bfill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].ffill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].bfill()
    return results

In [ ]:
processed_res = get_processed_results('ford')

In [ ]:
processed_res.dropna(subset=['soh', 'odometer']).shape

In [ ]:
px.scatter(processed_res, x='date', y='soh', color='vin')

In [ ]:
def get_processed_results_full(brand:str) -> DF:
    logger.info(f"{'Processing ' + brand + ' results.':=^{50}}")
    results =  (
        raw_res
        # Some raw estimations may have inf values, this will make mask_out_outliers_by_interquartile_range and force_monotonic_decrease fail
        # So we replace them by NaNs.
        .assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan))
        .sort_values(["vin", "date"])
        .pipe(make_charge_levels_presentable)
        .eval(SOH_FILTER_EVAL_STRINGS[brand])
        .pipe(agg_results_by_update_frequency)
        .groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle, include_groups=False)
        .reset_index(level=0)
        #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"])
    )
    results["soh"] = results.groupby("vin", observed=True)["soh"].ffill()
    results["soh"] = results.groupby("vin", observed=True)["soh"].bfill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].ffill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].bfill()
    return results

In [ ]:
processed_res_full = get_processed_results_full('ford')
processed_res_full.dropna(subset=['soh', 'odometer']).shape

In [ ]:
px.scatter(processed_res_full, x='odometer', y='soh', color='vin')

Il semblerait que les dégradations d'une semaine à l'autre sont > .003.
Au vu du nombre de point mesuré peut être que réutiliser l'ancienne meéthode serait plus logique 

In [ ]:
def make_soh_presentable_per_vehicle_old(df:DF) -> DF:
    if df["soh"].isna().all():
        return df
    if df["soh"].count() > 3:
        outliser_mask = mask_out_outliers_by_interquartile_range(df["soh"])
        assert outliser_mask.any(), f"There seems to be only outliers???:\n{df['soh']}."
        df = df[outliser_mask].copy()
    if df["soh"].count() >= 2:
        df["soh"] = force_monotonic_decrease(df["soh"])
    return df


def get_processed_results_old(brand:str) -> DF:
    logger.info(f"{'Processing ' + brand + ' results.':=^{50}}")
    results =  (
        raw_res
        # Some raw estimations may have inf values, this will make mask_out_outliers_by_interquartile_range and force_monotonic_decrease fail
        # So we replace them by NaNs.
        .assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan))
        .sort_values(["vin", "date"])
        .pipe(make_charge_levels_presentable)
        .eval(SOH_FILTER_EVAL_STRINGS[brand])
        .pipe(agg_results_by_update_frequency)
        .groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle_old, include_groups=False)
        .reset_index(level=0)
        #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"])
    )
    results["soh"] = results.groupby("vin", observed=True)["soh"].ffill()
    results["soh"] = results.groupby("vin", observed=True)["soh"].bfill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].ffill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].bfill()
    return results

In [ ]:
processed_res_old = get_processed_results_old('ford')
px.scatter(processed_res_old, x='date', y='soh', color='vin')

l'ancienne méthode retourne quelque chose de constant.

In [ ]:
dbeaver_df.vin.unique()

In [ ]:
vin_test = raw_res[raw_res['vin']=="WF0TK1E79MMA47996"].copy()
px.scatter(vin_test, x='date', y='soh')

In [ ]:
vin_test_replace_nan = vin_test.assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan)).sort_values(["vin", "date"])


In [ ]:
px.scatter(vin_test_replace_nan, x='odometer', y='soh')

In [ ]:
vin_test_replace_nan.shape

In [ ]:
vin_test_charge = vin_test_replace_nan.pipe(make_charge_levels_presentable)

In [ ]:
vin_test_charge.shape

In [ ]:
px.scatter(vin_test_charge, x='odometer', y='soh')

In [ ]:
vin_test_filter = vin_test_charge.eval(SOH_FILTER_EVAL_STRINGS['ford'])

In [ ]:
vin_test_filter.shape

In [ ]:
vin_test_median = vin_test_filter.pipe(agg_results_by_update_frequency)

In [ ]:
px.scatter(vin_test_median, x='odometer', y='soh')

In [ ]:
vin_test_prod = (vin_test_median.groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle, include_groups=False)
        .reset_index(level=0)
        #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"]))

In [ ]:
px.scatter(vin_test_prod, x='odometer', y='soh')

In [ ]:
def force_decay_bis(df, window_size=3, max_drop=0.003):
    """
    Génère une série strictement décroissante à partir d'une liste de valeurs en :
      - Calcule d'abord une moyenne mobile,
      - En choisissant un point de départ proche du maximum (pour refléter les meilleures valeurs),
      - Puis en s'assurant qu'entre deux points consécutifs, la diminution ne dépasse pas max_drop,
        et que la série ne stagne pas sur plus de 2 points.
    """
    # Calcul de la moyenne mobile
    smoothed = df['soh'].rolling(window=window_size, min_periods=2).mean().values
    # récupère l'odomètre pour la dcroissance forcé.
    odometer = df['odometer'].ffill().copy().values
    n = len(smoothed)
    
    # Pour le démarrage, on part du maximum de la série lissée / pas forcéement optimal
    odometer_fill = [odometer[1]]
    output = [max(smoothed[1:])]
    for i in range(1, n):
        candidate = smoothed[i]
        prev = output[-1]
        
        # Si  candidate >= à la précédente, on la force à être légèrement plus basse en se basant sur l'odomètre
        if candidate >= prev:
            epsilon = (odometer[i] - odometer[i-1]) * 1e-7
            candidate = prev - epsilon
            
        # Vérifier que le drop n'est pas trop important
        drop = prev - candidate
        if drop > max_drop:
            if odometer[i] - odometer_fill[-1] < 1000:
                candidate = prev
            else:
                candidate = prev - max_drop
        output.append(candidate)
        odometer_fill.append(odometer[i])
        
    return output

def make_soh_presentable_per_vehicle_bis(df:DF) -> DF:
    if df["soh"].isna().all():
        return df
    if df["soh"].count() > 3:
        outliser_mask = mask_out_outliers_by_interquartile_range(df["soh"])
        assert outliser_mask.any(), f"There seems to be only outliers???:\n{df['soh']}."
        df = df[outliser_mask].copy()
    if df["soh"].count() >= 2:
        df["soh"] = force_decay_bis(df[["soh", "odometer"]])
    return df

In [ ]:
vin_test_prod_bis = (vin_test_median.groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle_bis, include_groups=False)
        .reset_index(level=0)
        #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"]))

In [ ]:
px.scatter(vin_test_prod_bis, x='odometer', y='soh')

In [ ]:
def get_processed_results_bis(brand:str) -> DF:
    logger.info(f"{'Processing ' + brand + ' results.':=^{50}}")
    results =  (
        raw_res
        # Some raw estimations may have inf values, this will make mask_out_outliers_by_interquartile_range and force_monotonic_decrease fail
        # So we replace them by NaNs.
        .assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan))
        .sort_values(["vin", "date"])
        .pipe(make_charge_levels_presentable)
        .eval(SOH_FILTER_EVAL_STRINGS[brand])
        .pipe(agg_results_by_update_frequency)
        .groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle_bis, include_groups=False)
        .reset_index(level=0)
        #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"])
    )
    results["soh"] = results.groupby("vin", observed=True)["soh"].ffill()
    results["soh"] = results.groupby("vin", observed=True)["soh"].bfill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].ffill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].bfill()
    return results

In [ ]:
test_all = get_processed_results_bis('ford')

In [ ]:
px.scatter(test_all, x='odometer', y='soh', color='vin')

In [ ]:
px.scatter(processed_res_full, x='odometer', y='soh', color='vin')

In [ ]:
WF0TK1E79MMA47996